# 0. Import required packages

In [ ]:
import os
import yaml
import math
import copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from reportlab.lib import colors
import tools

In [ ]:
from massibo_ana.core.WaveformSet import WaveformSet
from massibo_ana.core.DarkNoiseMeas import DarkNoiseMeas
from massibo_ana.postprocess.PDFGenerator import PDFGenerator
import massibo_ana.utils.search as search
import massibo_ana.utils.custom_exceptions as cuex

# 1. Load the input parameters and change the WD to the workspace

In [ ]:
# Write here the ABSOLUTE path to the input parameters file
input_parameters_filepath = ''

with open(
        input_parameters_filepath,
        "r",
        encoding="utf-8"
    ) as file:
        
        params = yaml.safe_load(file)

os.chdir(params['workspace_dir'])
load_dir = "load/"
summary_dir = "summary/" 

# 2. Read the waveforms, rebin, compute baseline and flip

In [ ]:
darknoisemeas_objects = tools.build_list_of_SiPMMeas_objects(
    load_dir,
    params['strips_to_analyze'],
    params['analyzable_marks'],
    is_gain_meas=False,
    verbose=params['verbose']
)

if len(darknoisemeas_objects) == 0:
    raise Exception(
        "Not a single DarkNoiseMeas object was found coming from "
        f"a board with a strip ID within {params['strips_to_analyze']} "
        f"and with a mark within {params['analyzable_marks']}."
    )

for dno in darknoisemeas_objects:
    
    dno.rebin(params['merged_points'])

    # The baseline needs to be computed prior to
    # integrating the waveforms during the integral filter
    dno.Waveforms.compute_first_peak_baseline(
    signal_fraction_for_median_cutoff=params['signal_fraction_for_median_cutoff']
    )

    if params['flip_about_baseline']:
        dno.Waveforms.flip_about_baseline()

# Order the waveform sets according to their strip IDs
darknoisemeas_objects = tools.order_list_of_SiPMMeas_objects(darknoisemeas_objects)

# 3. Start the PDF report

In [ ]:
if params['generate_report']:

    pdf_chunks_filepaths_generator = PDFGenerator.PDF_chunk_filepath(
        summary_dir,
        params['report_output_filename']
    )
    
    aux = next(pdf_chunks_filepaths_generator)
    pdf_reports_filepaths = [aux]
    report_pdf = PDFGenerator(aux)

# 4. Apply offline filter (WaveformSet.integral_filter)

In [ ]:
# The analysis reliability parameter is set to 3 by default,
# which is the maximum reliability level. If after the
# offline filter, the number of waveforms for a certain SiPM
# is less than the minimum_number_of_well_formed_waveforms
# parameter, the reliability parameter for such SiPM is set
# to 0. If the analysis failed (particularly, the 0-PE and 1-PE
# amplitude-peaks fit, required for the 0.5- and 1.5-PE levels
# computation) the reliability is set to 1. If after the bursts
# purge, the number of waveforms for a certain SiPM is less
# than the minimum_number_of_non_burst_waveforms parameter, the
# reliability parameter for such SiPM is set to 2.
# Long story short, if the analysis breaks in the filter (first
# stage), the reliability is set to 0. If it breaks in the fit
# (second stage), the reliability is set to 1. If it breaks in
# the bursts purge (third stage), the reliability is set to 2.
# If everything went OK, the reliability remains set to 3.
analysis_reliability = [3] * len(darknoisemeas_objects)

filtered_dnos = []
for i in range(len(darknoisemeas_objects)):

    if params['verbose']:
        print(
            f"Deep-copying the {i+1}-th "
            f"(/{len(darknoisemeas_objects)}) SiPM data "
            f"- {darknoisemeas_objects[i].get_title()}"
        )

    # The filtered version of darknoisemeas_objects[i] is filtered_dnos[i]
    filtered_dnos.append(copy.deepcopy(darknoisemeas_objects[i]))

noise_idcs = {}
for idx in range(len(filtered_dnos)):

    # dno is a view to filtered_dnos[idx]
    dno = filtered_dnos[idx]

    if params['verbose']:
        print(
            f"Filtering the {idx+1}-th (/"
            f"{len(filtered_dnos)}) SiPM data - {dno.get_title()}"
        )

    wvfs_no = len(dno.Waveforms)
    # Assuming every waveform in dno.Waveforms
    # has the same number of data points
    points_per_wvf = len(dno.Waveforms[0].Signal)

    # Assuming that the Time attribute is the same
    # for all waveforms in the same DarkNoiseMeas object
    aux, _ = search.find_nearest_neighbour(
        dno.Waveforms[0].Time,
        params['window_start_s']
    )
        
    noise_idcs[idx] = dno.Waveforms.filter(
        WaveformSet.integral_filter,
        # This one is given as an arg to the filter function
        params['minimum_integral'],
        return_idcs=True,
        purge=True,
        ask_for_confirmation=False,
        # Given as a kwarg to the filter function
        i0=aux,
        # delta_t=dno.Waveforms[i].find_beginning_of_rise(
        #     tolerance=params['tolerance_for_beginning_of_rise'],
        #     return_iterator=False
        # )+params['delta_t_from_beginning_of_rise'],
        delta_t=params['integral_window_width_s'],
        also_return_integral=False
    )

    if params['verbose']:
        print(
            "Eliminated "
            f"{round(100.*len(noise_idcs[idx])/len(darknoisemeas_objects[idx].Waveforms), ndigits=2)}"
            "% of the fast frames"
        )
    
    if len(dno.Waveforms) < params['minimum_number_of_well_formed_waveforms']:
        analysis_reliability[idx] = 0

        if params['verbose']:
            print(
                f"WARNING: Only {len(dno.Waveforms)} waveforms "
                f"(< {params['minimum_number_of_well_formed_waveforms']}) "
                "were left after filtering the noise-induced triggers. "
                "The analysis for this SiPM has been tagged as 0-reliable."
            )
    

# 5. Run the peak detection algorithm

In [ ]:
for idx in range(len(filtered_dnos)):

    minimum_peak_width_in_samples = math.floor(
        params['minimum_peak_width_in_s']/(filtered_dnos[idx].Sampling_ns*1e-9)
    )

    if params['verbose']:
        print(
            f"Detecting peaks for the {idx+1}-th (/"
            f"{len(filtered_dnos)}) SiPM data - {filtered_dnos[idx].get_title()}"
        )

    filtered_dnos[idx].Waveforms.find_peaks(
        height=params['minimal_height_wrt_baseline_in_AU'],
        prominence=params['minimum_peaks_prominence'],
        width=minimum_peak_width_in_samples,
        rel_height=params['rel_height_for_peak_width'],
    )

# 6. Report results of offline filter and the peak-detection algorithm

In [ ]:
if params['generate_report']:

    # Number of accepted-waveforms persistency plots
    # It is also the number of rejected-waveforms
    # persistency plots. Setting this parameter to a value
    # other than 2 would break the report PDF layout.
    steps = 2

    # Number of grid-plots per SiPM. Setting this
    # parameter to a value other than 2 would break
    # the report PDF layout.
    canvases_to_show_per_sipm = 2

    # Number of rows and columns in each grid-plot.
    # Setting these parameter to a value other than
    # (3, 5) would break the report PDF layout.
    nrows = 3
    ncols = 5

    for idx in range(len(darknoisemeas_objects)):

        if params['verbose']:
            print(
                "Reporting the offline-filter results for "
                f"the {idx+1}-th (/{len(filtered_dnos)}) SiPM "
                f"data - {filtered_dnos[idx].get_title()}"
            )

        report_pdf.add_text(
            # filtered_dnos preserve the ordering of darknoisemeas_objects
            f"Iterator {idx}, {darknoisemeas_objects[idx].get_title()}",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.95,
            max_width_frac=0.9,
            font_size=params['title_font_size'],
            horizontally_center=True
        )

        report_pdf.add_text(
            f"Offline filter results",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.88,
            max_width_frac=0.9,
            font_size=params['subtitle_font_size'],
            horizontally_center=True
        )

        report_pdf.add_text(
            f"Filtered out {len(noise_idcs[idx])} out of "
            f"{len(darknoisemeas_objects[idx].Waveforms)} "
            f"fast frames (i.e. {round(100.*len(noise_idcs[idx])/len(darknoisemeas_objects[idx].Waveforms), ndigits=2)}"
            f"% of the fast frames), leaving {len(filtered_dnos[idx].Waveforms)} waveforms for the analysis",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.84,
            max_width_frac=0.9,
            font_size=params['text_font_size'],
            horizontally_center=True
        )

        if analysis_reliability[idx] < 1:
            report_pdf.add_text(
                f"WARNING: The analysis for this SiPM has "
                "been tagged as 0-reliable because the filter "
                f"left less than {params['minimum_number_of_well_formed_waveforms']}"
                " waveforms",
                horizontal_pos_frac=0.40,
                vertical_pos_frac=0.89,
                max_width_frac=0.6,
                font_size=params['text_font_size']-2,
                font_color=colors.red,
                horizontally_center=False
            )

        dno = filtered_dnos[idx]
        for i in range(steps):

            aux = dno.Waveforms.plot(
                # Plot the filtered dataset
                wvfs_to_plot=list(
                    range(
                        i*params['wvfs_per_step'],
                        (i+1)*params['wvfs_per_step']
                    )
                ) if (i+1)*params['wvfs_per_step'] <= len(dno.Waveforms) else
                list(
                    range(
                        i*params['wvfs_per_step'],
                        len(dno.Waveforms)
                    )
                ),
                plot_peaks=False,
                fig_title=f"Accepted frames ({params['wvfs_per_step']} samples)"
                if (i+1)*params['wvfs_per_step'] <= len(dno.Waveforms) else
                f"Accepted frames ({len(dno.Waveforms) - (i * params['wvfs_per_step'])} samples)",
                mode='superposition',
                title_fontsize=params['title_font_size'],
                wvf_linewidth=params['persistency_wvf_linewidth'],
                subtract_baseline=True,
                show_plots=False
            )
            report_pdf.add_plot(
                aux[0],
                horizontal_pos_frac=0.01 if i == 0 else 0.26,
                vertical_pos_frac=0.45,
                plot_width_wrt_page_width=0.25,
                horizontally_center=False
            )

        dno = darknoisemeas_objects[idx]
        idcs_to_plot = noise_idcs[idx]
        for i in range(steps):

            aux = dno.Waveforms.plot(
                # Plot the selected indices from the unfiltered dataset
                wvfs_to_plot=idcs_to_plot[i*params['wvfs_per_step']:(i+1)*params['wvfs_per_step']]
                if (i+1)*params['wvfs_per_step'] <= len(idcs_to_plot) else
                idcs_to_plot[i*params['wvfs_per_step']:],
                plot_peaks=False,
                fig_title=f"Rejected frames ({params['wvfs_per_step']} samples)"
                if (i+1)*params['wvfs_per_step'] <= len(idcs_to_plot) else
                f"Rejected frames ({len(idcs_to_plot) - (i * params['wvfs_per_step'])} samples)",
                mode='superposition',
                title_fontsize=params['title_font_size'],
                wvf_linewidth=params['persistency_wvf_linewidth'],
                subtract_baseline=True,
                show_plots=False
            )
            
            report_pdf.add_plot(
                aux[0],
                horizontal_pos_frac=0.51 if i == 0 else 0.76,
                vertical_pos_frac=0.45,
                plot_width_wrt_page_width=0.25,
                horizontally_center=False
            )
            
            if (i+1)*params['wvfs_per_step'] > len(idcs_to_plot):
                break

        if params['verbose']:
            print(
                "Reporting the peak-detection results for "
                f"the {idx+1}-th (/{len(filtered_dnos)}) "
                f"SiPM data - {filtered_dnos[idx].get_title()}"
            )

        report_pdf.add_text(
            f"Some peak-detection examples",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.63,
            max_width_frac=0.9,
            font_size=params['subtitle_font_size'],
            horizontally_center=True
        )

        if len(filtered_dnos[idx].Waveforms) > 0:

            aux = filtered_dnos[idx].Waveforms.plot(
                wvfs_to_plot=min(
                    canvases_to_show_per_sipm * nrows * ncols,
                    len(filtered_dnos[idx].Waveforms)
                ), 
                plot_peaks=True,
                wvf_linewidth=0.2,
                x0=[],
                y0=[],
                randomize=True,
                fig_title=None,
                mode='grid',
                nrows=nrows,
                ncols=ncols,
                fig_width=10.,
                fig_height=5.0,
                show_plots=False
            )

            for i in range(
                min(
                    canvases_to_show_per_sipm,
                    len(aux)
                )
            ):
                report_pdf.add_plot(
                    aux[i],
                    horizontal_pos_frac=None,
                    # Assuming the number of canvases
                    # to show per SiPM is 2
                    vertical_pos_frac={
                        0: 0.18,
                        1: -0.1
                    }[i],
                    plot_width_wrt_page_width=0.8,
                    horizontally_center=True
                )
        else:
            report_pdf.add_text(
                "WARNING: The offline filter left no waveforms "
                "for this SiPM. There are no peak-detection "
                "results",
                horizontal_pos_frac=None,
                vertical_pos_frac=0.35,
                max_width_frac=0.9,
                font_size=params['text_font_size'],
                font_color=colors.red,
                horizontally_center=True
            )
    
        started_a_new_pdf, aux = PDFGenerator.smart_close_page(
            report_pdf,
            params['max_pages_per_pdf_chunk'],
            pdf_chunks_filepaths_generator
        )

        if started_a_new_pdf:
            del report_pdf
            report_pdf = aux
            pdf_reports_filepaths.append(report_pdf.OutputFilepath)

# 7. Run the analysis

In [ ]:
for idx in range(len(filtered_dnos)):

    if analysis_reliability[idx] < 1:
        if params['verbose']:
            print(
                f"Skipping the levels analysis for the {idx+1}-th "
                f"(/{len(filtered_dnos)}) (0-reliable) SiPM data - "
                f"{filtered_dnos[idx].get_title()}"
            )
        continue
    else:
        if params['verbose']:
            print(
                f"Computing the 0.5- and 1.5-PE levels for the "
                f"{idx+1}-th (/{len(filtered_dnos)}) SiPM data - "
                f"{filtered_dnos[idx].get_title()}"
            )
    try:
        filtered_dnos[idx].analyze(
            peaks_to_detect=2,
            bins_no=params['amplitudes_histogram_bins_no'],
            starting_fraction=params['starting_fraction'],
            step_fraction=params['step_fraction'],
            # You may need to tune this one to avoid labelling
            # as an ill-formed case a SiPM with very low XTP 
            # (so low that the second peak in the amplitude 
            # histogram is very small so as to be ruled out 
            # by the prominence cut)
            minimal_prominence_wrt_max=params['minimal_prominence_wrt_max'],
            std_no=params['gaussian_fit_std_no'],
            timedelay_cut=params['timedelay_cut']
        )
    except RuntimeError:
        try: 
            print(
                f"WARNING: Got a RuntimeError when processing "
                f"the {idx+1}-th SiPM data. Now trying to reanalyze "
                f"it using a softer prominence (minimal_prominence_wrt_max = "
                f"{params['minimal_prominence_wrt_max']} -> "
                f"{params['minimal_prominence_wrt_max'] - (0.25 * params['minimal_prominence_wrt_max'])})"
                " requirement."
            )
            filtered_dnos[idx].analyze(
                peaks_to_detect=2,
                bins_no=params['amplitudes_histogram_bins_no'],
                starting_fraction=params['starting_fraction'],
                step_fraction=params['step_fraction'],
                # Decrease the required prominence for the peaks
                # to be detected by a 25% of the original value
                minimal_prominence_wrt_max=params['minimal_prominence_wrt_max'] 
                - 0.25 * params['minimal_prominence_wrt_max'],
                std_no=params['gaussian_fit_std_no'],
                timedelay_cut=params['timedelay_cut']
            )

            print(
                f"Re-analysis of the {idx+1}-th SiPM data "
                "executed normally."
            )

        except RuntimeError:
            print(
                f"WARNING: Still got a RuntimeError when re-processing "
                f"the {idx+1}-th SiPM data. The analysis for this "
                "SiPM has been tagged as 1-reliable."
            )
            analysis_reliability[idx] = 1
            continue

    except cuex.RestrictiveTimedelay:

        print(
            f"WARNING: Got a cuex.RestrictiveTimedelay exception "
            f"when processing the {idx+1}-th SiPM data. Now "
            "reanalyzing it using a softer cut for the timedelay "
            f"(timedelay_cut = {params['timedelay_cut']} -> {params['timedelay_cut']/10.})."
        )
        
        filtered_dnos[idx].analyze(
            peaks_to_detect=2,
            bins_no=params['amplitudes_histogram_bins_no'],
            starting_fraction=params['starting_fraction'],
            step_fraction=params['step_fraction'],
            # Decrease the required prominence for the peaks
            # to be detected by a 25% of the original value
            minimal_prominence_wrt_max=params['minimal_prominence_wrt_max'],
            std_no=params['gaussian_fit_std_no'],
            timedelay_cut=params['timedelay_cut']/10.
        )

        print(
            f"Re-analysis of the {idx+1}-th SiPM data "
            "executed normally."
        )

    # When scipy.optimize.curve_fit() throws an exception message
    # a la "Improper input: func input vector length N=3 must not exceed
    # func output vector length M=1", SiPMMeas.piecewise_gaussian_fits()
    # raises a cuex.NotEnoughFitSamples exception. A cause for this is
    # that the number of points-to-fit given to scipy.optimize.curve_fit()
    # is smaller than than the number of fitting parameters. In our
    # particular context, this could have happened because the value
    # given to the std_no parameter of DarkNoiseMeas.analyze(), which is
    # eventually given to SiPMMeas.piecewise_gaussian_fits() is so small
    # that the number of points-to-fit is smaller than the number of
    # fitting parameters. 
    except cuex.NotEnoughFitSamples:

        print(
            f"WARNING: Got a cuex.NotEnoughFitSamples exception "
            f"when processing the {idx+1}-th SiPM data. Now reanalyzing "
            f"it using a bigger fitting range (std_no = {params['gaussian_fit_std_no']} -> "
            f"{params['gaussian_fit_std_no']+1.})."
        )
        
        filtered_dnos[idx].analyze(
            peaks_to_detect=2,
            bins_no=params['amplitudes_histogram_bins_no'],
            starting_fraction=params['starting_fraction'],
            step_fraction=params['step_fraction'],
            minimal_prominence_wrt_max=params['minimal_prominence_wrt_max'],
            std_no=params['gaussian_fit_std_no']+1.,
            timedelay_cut=params['timedelay_cut']
        )

# 8. Report the analysis results

In [ ]:
if params['generate_report']:

    # The values given to these parameters fit
    # the layout of the report PDF.
    # Number of rows and columns in each grid-plot
    rows_per_canvas = 2
    cols_per_canvas = 3

    axes_per_canvas = rows_per_canvas*cols_per_canvas
    objects_to_plot = len(filtered_dnos)
    canvases_no = math.ceil(objects_to_plot/axes_per_canvas)

    figs, axes = [], []
    for i in range(canvases_no):
        aux_fig, aux_ax = plt.subplots(
            nrows=rows_per_canvas, 
            ncols=cols_per_canvas
        )
        aux_fig.set_figheight(6)
        aux_fig.set_figwidth(10)
        figs.append(aux_fig)
        axes.append(aux_ax)
        aux_fig.tight_layout()

    for i in range(len(filtered_dnos)):

        if analysis_reliability[i] < 1:
            if params['verbose']:
                print(
                    "Skipping the timedelay vs. amplitude plot "
                    f"for the {i+1}-th (/{len(filtered_dnos)}) "
                    f"(0-reliable) SiPM data - {filtered_dnos[i].get_title()}"
                )
            continue
        else:
            if params['verbose']:
                print(
                    "Plotting timedelay vs. amplitude for the "
                    f"{i+1}-th (/{len(filtered_dnos)}) SiPM "
                    f"data - {filtered_dnos[i].get_title()}"
                )
        
        idx = i%axes_per_canvas
        j, k = int(idx//cols_per_canvas), int(idx%cols_per_canvas)
        current_fig = figs[int(i//axes_per_canvas)]
        current_axes = axes[int(i//axes_per_canvas)][j, k]
            
        filtered_dnos[i].plot_timedelay_vs_amplitude(   
            current_axes,
            mode='hist2d', 
            nbins=150,
            axes_title=darknoisemeas_objects[i].get_title(abbreviate=True),
            plot_half_a_pe_level=True,
        )

        try:
            WaveformSet.set_custom_labels_visibility(current_axes, j, k, rows_per_canvas)
        except AttributeError:
            continue
        
        # Are you getting a TypeError from scipy.optimize.curve_fit? 
        # scipy.signal.find_peaks() might be detecting mini-peaks 
        # which are made up of just one or two samples. In this case, 
        # fitting a 3-parameters gaussian to 1 or 2 samples is 
        # not (mathematically) well constrained. To filter out these
        # mini-peaks, tune up minimal_prominence_wrt_max.


    for i in range(canvases_no):
        
        if params['verbose']:
            print(
                "Reporting the levels-computation results "
                f"for the [{(i * axes_per_canvas) + 1}-"
                f"{(i+1)*axes_per_canvas}] (/{len(filtered_dnos)}) "
                "SiPM data"
            )

        # Two plots per page
        if i%2 == 0:

            # If this is the first plot we are adding
            # to this page, then add the title and the
            # first plot
            report_pdf.add_text(
                f"0.5- and 1.5-PE computation results",
                horizontal_pos_frac=None,
                vertical_pos_frac=0.95,
                max_width_frac=0.9,
                font_size=params['title_font_size'],
                horizontally_center=True
            )

            figs[i].tight_layout()
            report_pdf.add_plot(
                figs[i],
                horizontal_pos_frac=None,
                vertical_pos_frac=0.35,
                plot_width_wrt_page_width=0.8,
                horizontally_center=True
            )

            # If it's the last plot we are adding,
            # even if it's the first one of the page,
            # then close the page and start a new one
            # for plots of other sections
            if i == canvases_no-1:
                report_pdf.close_page_and_start_a_new_one() 

        else: # i%2 == 1

            # If it's the second plot we are adding
            # to this page, then put it below the
            # first one and go to a new page
            figs[i].tight_layout()
            report_pdf.add_plot(
                figs[i],
                horizontal_pos_frac=None,
                vertical_pos_frac=0.0,
                plot_width_wrt_page_width=0.8,
                horizontally_center=True
            )

            started_a_new_pdf, aux = PDFGenerator.smart_close_page(
                report_pdf,
                params['max_pages_per_pdf_chunk'],
                pdf_chunks_filepaths_generator
            )

            if started_a_new_pdf:
                del report_pdf
                report_pdf = aux
                pdf_reports_filepaths.append(report_pdf.OutputFilepath)

# 9. Compute DCR, XTP and APP

In [ ]:
i_DCR = []
i_XTP = []
i_APP = []

for i in range(len(filtered_dnos)):

    if analysis_reliability[i] < 2:
        if params['verbose']:
            print(
                "Skipping the DCR, XTP and APP computation "
                f"for the {i+1}-th (/{len(filtered_dnos)}) "
                f"(0/1-reliable) SiPM data - {filtered_dnos[i].get_title()}"
            )
        continue
    else:
        if params['verbose']:
            print(
                "Computing the DCR, XTP and APP for the "
                f"{i+1}-th (/{len(filtered_dnos)}) SiPM "
                f"data - {filtered_dnos[i].get_title()}"
            )

    i_DCR.append((
        i, 
        filtered_dnos[i].get_dark_count_rate_in_mHz_per_mm2(    # May throw a TypeError, characterize it
            params['sipm_sensitive_surface_area_in_mm2']
        )
    ))

    i_XTP.append((
        i,
        filtered_dnos[i].get_cross_talk_probability()   # May throw a cuex.NoAvailableData, characterize it
    ))

    i_APP.append((
        i,
        filtered_dnos[i].get_after_pulse_probability()  # May throw a TypeError, characterize it
    ))

DCR = [i_DCR[i][1] for i in range(len(i_DCR))]
XTP = [i_XTP[i][1] for i in range(len(i_XTP))]
APP = [i_APP[i][1] for i in range(len(i_APP))]

# 10. Purge from bursts

In [ ]:
purged_darknoisemeas_objects = []
for i in range(len(darknoisemeas_objects)):
    # Note that, in order to compute the purged DN objects,
    # what we are purging is the already-filtered objects
    # Also note that, by construction of filtered_dnos 
    # (some cells above), the purged version of 
    # darknoisemeas_objects[i] is purged_darknoisemeas_objects[i]

    if analysis_reliability[i] < 2:
        if params['verbose']:
            print(
                f"Skipping the purging of the {i+1}-th "
                f"(/{len(filtered_dnos)}) (0/1-reliable) "
                f"SiPM data - {filtered_dnos[i].get_title()}"
            )

        # Add a None entry for the sake of preserving
        # the ordering of the darknoisemeas_objects
        purged_darknoisemeas_objects.append(None)

    else:
        if params['verbose']:
            print(
                f"Purging from bursts the {i+1}-th "
                f"(/{len(filtered_dnos)}) SiPM data "
                f"- {filtered_dnos[idx].get_title()}"
            )

        purged_darknoisemeas_objects.append(
            DarkNoiseMeas.purge_bursts(
                filtered_dnos[i],
                min_events_no=params['min_consecutive_peaks_for_burst'],
                timedelay_threshold_in_s=params['max_timedelay_for_consecutive_peak_in_s']
            )
        )

        if len(purged_darknoisemeas_objects[-1].Waveforms) < params['minimum_number_of_non_burst_waveforms']:

            analysis_reliability[i] = 2

            if params['verbose']:
                print(f"WARNING: Only {len(purged_darknoisemeas_objects[-1].Waveforms)} "
                      f"waveforms (< {params['minimum_number_of_non_burst_waveforms']}) "
                      f"were left after purging the busts. The analysis for this "
                      "SiPM has been tagged as 2-reliable.")

# 11. Generate output dataframe

In [ ]:
data = None
fIsFirst = True

# Iterating over filtered_dnos
for i in range(len(filtered_dnos)):

    aux = filtered_dnos[i].output_summary(        
        params['sipm_sensitive_surface_area_in_mm2'],
        additional_entries= {
            "merged_points": params['merged_points'],
            "sipm_sensitive_surface_area_in_mm2": params['sipm_sensitive_surface_area_in_mm2'],
            "signal_fraction_for_median_cutoff": params['signal_fraction_for_median_cutoff'],
            "minimal_height_wrt_baseline_in_AU": params['minimal_height_wrt_baseline_in_AU'],
            "minimum_peaks_prominence": params['minimum_peaks_prominence'],
            "minimum_peak_width_in_s": params['minimum_peak_width_in_s'],
            "rel_height_for_peak_width": params['rel_height_for_peak_width'],
            "starting_fraction": params['starting_fraction'],
            "step_fraction": params['step_fraction'],
            "minimal_prominence_wrt_max": params['minimal_prominence_wrt_max'],
            "gaussian_fit_std_no": params['gaussian_fit_std_no'],
            "amplitudes_histogram_bins_no": params['amplitudes_histogram_bins_no'],
            "timedelay_cut": params['timedelay_cut'],
            "minimum_integral": params['minimum_integral'],
            "integral_window_width_s": params['integral_window_width_s'],
            "min_consecutive_peaks_for_burst": params['min_consecutive_peaks_for_burst'],
            "max_timedelay_for_consecutive_peak_in_s": params['max_timedelay_for_consecutive_peak_in_s'],
            "burstless_DC#": purged_darknoisemeas_objects[i].get_dark_counts_number()
            if analysis_reliability[i] > 2 else float('nan'),
            "burstless_DCR_mHz_per_mm2": purged_darknoisemeas_objects[i].get_dark_count_rate_in_mHz_per_mm2(
                params['sipm_sensitive_surface_area_in_mm2']
            ) if analysis_reliability[i] > 2 else float('nan'),
            "burstless_XTP": purged_darknoisemeas_objects[i].get_cross_talk_probability()
            if analysis_reliability[i] > 2 else float('nan'),
            "burstless_APP": purged_darknoisemeas_objects[i].get_after_pulse_probability()
            if analysis_reliability[i] > 2 else float('nan'),
            "is_filtered": True,
            "filter_type": 'integral',
            "analysis_reliability": analysis_reliability[i]
        },
        folderpath=None,
        include_analysis_results=True if analysis_reliability[i] > 1 else False,
        overwrite=params['json_overwrite'],
        indent=params['indent'],
        verbose=params['verbose'])
    
    if fIsFirst:
        # Convert the model data to a dictionary of lists
        data = {key: [value] for key, value in aux.items()}
        fIsFirst = False
    else:
        for key in data.keys():
            data[key].append(aux[key])

output_dataframe = pd.DataFrame(data)

if params['verbose']:
    print(
        f"Saving the output dataframe to "
        f"{os.path.abspath(summary_dir)}/{params['output_dataframe_filename']}"
    )

output_dataframe.to_csv(
    os.path.join(
        summary_dir,
        params['output_dataframe_filename']+'.csv'
    )
)

output_dataframe.to_pickle(
    os.path.join(
        summary_dir,
        params['output_dataframe_filename']+'.pkl'
    )
)

# 12. Report the DCR, XTP and APP resulting distributions

In [ ]:
field_to_show = [
    'DCR_mHz_per_mm2',
    'XTP',
    'APP'
]
table_ndigits = {
    'DCR_mHz_per_mm2': 3,
    'XTP': 2,
    'APP': 2
}

# For mean and std computation
samples_wo_outliers = {
    'DCR_mHz_per_mm2': np.array([
        sample for sample in DCR 
        if sample <= tools.thresholds['DCR_mHz_per_mm2']['threshold']]),
    'XTP': np.array([
        sample for sample in XTP 
        if sample <= tools.thresholds['XTP']['threshold']]),
    'APP': np.array([
        sample for sample in APP 
        if sample <= tools.thresholds['APP']['threshold']])
}
red_lightness = 0.7
colour_decide = {
    'DCR_mHz_per_mm2': lambda val : (0.8, 0.8, 0.) if math.isnan(val)
    else (
        (1., 0., 0.)
        if val > tools.thresholds['DCR_mHz_per_mm2']['threshold']
        else ((1.,red_lightness,red_lightness) if val > tools.thresholds['DCR_mHz_per_mm2']['pre_threshold']
        else 'white')
    ),
    'XTP': lambda val : (0.8, 0.8, 0.) if math.isnan(val)
    else (
        (1., 0., 0.)
        if val > tools.thresholds['XTP']['threshold']
        else ((1.,red_lightness,red_lightness) if val > tools.thresholds['XTP']['pre_threshold']
        else 'white')
    ),

    'APP': lambda val : (0.8, 0.8, 0.) if math.isnan(val)
    else (
        (1., 0., 0.)
        if val > tools.thresholds['APP']['threshold']
        else ((1.,red_lightness,red_lightness) if val > tools.thresholds['APP']['pre_threshold']
        else 'white')
    )
}

In [ ]:
if params['generate_report']:

    fig, axes = plt.subplots(
        ncols=3,
        figsize=(15, 5)
    )

    tools.plot_histogram(
        axes[0],
        np.array(DCR),
        bins=params['resulting_distributions_nbins'],
        hist_range=(0., 2.*tools.thresholds['DCR_mHz_per_mm2']['threshold']),
        xlabel=r'DCR (mHz/mm$^2$)',
        ylabel='Hits', 
        figtitle=r"$\overline{\text{DCR}}=$"+f"{np.round(np.mean(samples_wo_outliers['DCR_mHz_per_mm2']), decimals=2)}, "+
        r"$\text{STD}=$"+f"{np.round(np.std(samples_wo_outliers['DCR_mHz_per_mm2']), decimals=2)}",
        fontsize=params['text_font_size'],
        yticks_step=1
    )
    axes[0].axvline(
        x=tools.thresholds['DCR_mHz_per_mm2']['threshold'],
        color=(1.0, 0.0, 0.0),
        linestyle='-',
        linewidth=params['vertical_thresholds_linewidth']
    )
    axes[0].axvline(
        x=tools.thresholds['DCR_mHz_per_mm2']['pre_threshold'],
        color=(1.0, red_lightness, red_lightness),
        linestyle='--',
        linewidth=params['vertical_thresholds_linewidth']
    )
    tools.plot_histogram( 
        axes[1],
        np.array(XTP),
        bins=params['resulting_distributions_nbins'],
        hist_range=(0., 2.*tools.thresholds['XTP']['threshold']),
        xlabel='X-Talk probability',
        ylabel='Hits', 
        figtitle=r"$\overline{\text{XTP}}=$"+f"{np.round(np.mean(samples_wo_outliers['XTP']), decimals=3)}, "+
        r"$\text{STD}=$"+f"{np.round(np.std(samples_wo_outliers['XTP']), decimals=3)}",
        fontsize=params['text_font_size'],
        yticks_step=1
    )
    axes[1].axvline(
        x=tools.thresholds['XTP']['threshold'],
        color=(1.0, 0.0, 0.0),
        linestyle='-',
        linewidth=params['vertical_thresholds_linewidth']
    )
    axes[1].axvline(
        x=tools.thresholds['XTP']['pre_threshold'],
        color=(1.0, red_lightness, red_lightness),
        linestyle='--',
        linewidth=params['vertical_thresholds_linewidth']
    )
    tools.plot_histogram(
        axes[2],
        np.array(APP),
        bins=params['resulting_distributions_nbins'],
        hist_range=(0., 2.*tools.thresholds['APP']['threshold']),
        xlabel='After pulse probability',
        ylabel='Hits', 
        figtitle=r"$\overline{\text{APP}}=$"+f"{np.round(np.mean(samples_wo_outliers['APP']), decimals=3)}, "+
        r"$\text{STD}=$"+f"{np.round(np.std(samples_wo_outliers['APP']), decimals=3)}",
        fontsize=params['text_font_size'],
        yticks_step=1
    )
    axes[2].axvline(
        x=tools.thresholds['APP']['threshold'],
        color=(1.0, 0.0, 0.0),
        linestyle='-',
        linewidth=params['vertical_thresholds_linewidth']
    )
    axes[2].axvline(
        x=tools.thresholds['APP']['pre_threshold'],
        color=(1.0, red_lightness, red_lightness),
        linestyle='--',
        linewidth=params['vertical_thresholds_linewidth']
    )

    if params['verbose']:
        print(
            "Reporting the DCR, XTP and APP "
            "distributions including all of the SiPMs"
        )

    report_pdf.add_text(
        f"DCR, XTP and APP resulting distributions",
        horizontal_pos_frac=None,
        vertical_pos_frac=0.95,
        max_width_frac=0.9,
        font_size=params['title_font_size'],
        horizontally_center=True
    )

    fig.tight_layout()
    report_pdf.add_plot(
        fig,
        horizontal_pos_frac=None,
        vertical_pos_frac=0.45,
        plot_width_wrt_page_width=0.99,
        horizontally_center=True
    )

    # Don't close the last page nor save the PDF
    # yet, because we will place some tables in it

# 13. Report the DCR, XTP and APP result-tables

In [ ]:
tables_vertical_pos_frac = {
    'DCR_mHz_per_mm2': 0.3,
    'XTP': 0.2,
    'APP': 0.1
}

if params['generate_report']:
    
    for variable in field_to_show:

        table = tools.strip_ID_vs_sipm_location_dataframe(
            output_dataframe, 
            variable, 
            significant_figures=table_ndigits[variable]
        )

        fig, ax = plt.subplots(figsize=(15, 2))
        ax.axis('off')
        ax.table(
            cellText=table.values, 
            colLabels=table.columns, 
            rowLabels=[' '+str(aux)+' ' for aux in range(1,7)], 
            colWidths = [0.06 for _ in table.columns],
            cellColours = [
                [
                    colour_decide[variable](
                        float(val)
                    ) for val in row
                ] for row in table.values
            ],
            cellLoc = 'center',
            loc='center'
        )
        ax.set_title(variable)
        
        report_pdf.add_plot(
            fig,
            horizontal_pos_frac=None,
            vertical_pos_frac=tables_vertical_pos_frac[variable],
            plot_width_wrt_page_width=0.99,
            horizontally_center=True
        )

    report_pdf.save()

# 14. Generate the cover

In [ ]:
if params['generate_report']:

    path_to_cover_pdf = os.path.join(
        summary_dir,
        'temp_cover.pdf'
    )

    cover_pdf = PDFGenerator(path_to_cover_pdf)

    cover_pdf.add_text(
        f"Darknoise analysis report",
        horizontal_pos_frac=None,
        vertical_pos_frac=0.95,
        max_width_frac=0.9,
        font_size=params['title_font_size'],
        horizontally_center=True
    )

    clustered_boards_string = tools.get_string_of_contiguously_clustered_integers(
        tools.cluster_integers_by_contiguity(list(output_dataframe.groupby('strip_ID').groups.keys()))
    )

    cover_pdf.add_text(
        f"Analyzed boards: {clustered_boards_string}",
        horizontal_pos_frac=None,
        vertical_pos_frac=0.90,
        max_width_frac=0.9,
        font_size=params['subtitle_font_size'],
        horizontally_center=True
    )

    cover_pdf.add_text(
        f"The results have been dumped to both, a CSV file "
        f"('{params['output_dataframe_filename']}.csv') and a pickle file "
        f"('{params['output_dataframe_filename']}.pkl'), saved alongside this"
        " report.", 
        horizontal_pos_frac=None,
        vertical_pos_frac=0.80,
        max_width_frac=0.9,
        font_size=params['text_font_size'],
        horizontally_center=True
    )

    # n_reliable_analyses[i] gives the number of SiPMs
    # with analysis_reliability[i] equal to n, where
    # n = 0, 1, 2
    n_reliable_analyses = {
        i: np.count_nonzero(np.array(analysis_reliability) == i)
        for i in range(3)
    }

    if sum(list(n_reliable_analyses.values())) == 0:
        cover_pdf.add_text(
            f"All of the analyses have been tagged as reliable",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.71,
            max_width_frac=0.9,
            font_size=params['text_font_size'],
            font_color=colors.green,
            horizontally_center=True
        )

    else:
        cover_pdf.add_text(
            f"A total of {n_reliable_analyses[0]} (resp. "
            f"{n_reliable_analyses[1]}, {n_reliable_analyses[2]}) "
            f"analyses, out of {len(darknoisemeas_objects)} - the "
            f"{round(100.*n_reliable_analyses[0]/len(darknoisemeas_objects), ndigits=2)} % "
            f"(resp. {round(100.*n_reliable_analyses[1]/len(darknoisemeas_objects), ndigits=2)} %, "
            f"{round(100.*n_reliable_analyses[2]/len(darknoisemeas_objects), ndigits=2)} %) - "
            "have been tagged as 0- (resp. 1-, 2-) reliable:",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.71,
            max_width_frac=0.9,
            font_size=params['text_font_size'],
            font_color=colors.red,
            horizontally_center=True
        )

        added_lines_so_far = tools.natural_numbers_generator()
        added_lines_so_far_in_this_page = 0
        fNewPage = False
        aux_starting_vertical_pos_frac = {
            True: 0.9, False: 0.67
        }

        for i in range(len(analysis_reliability)):
            if analysis_reliability[i] < 3:
                vertical_pos_frac = \
                    aux_starting_vertical_pos_frac[fNewPage] \
                        - (0.02 * (added_lines_so_far_in_this_page))

                if vertical_pos_frac < 0.1:
                    # Most of the times, the cover PDF will encompass
                    # just one page. For very ill-formed cases, it can
                    # go up to some pages (less than 10). That's why
                    # I am not using the smart-close-page method here.
                    cover_pdf.close_page_and_start_a_new_one()
                    fNewPage = True

                    added_lines_so_far_in_this_page = 0
                    vertical_pos_frac = \
                        aux_starting_vertical_pos_frac[fNewPage]

                cover_pdf.add_text(
                    f"{next(added_lines_so_far)}) {darknoisemeas_objects[i].get_title()}",
                    horizontal_pos_frac=None,
                    vertical_pos_frac=vertical_pos_frac,
                    max_width_frac=0.9,
                    font_size=params['text_font_size']-4.,
                    font_color={
                        0: colors.red,
                        1: colors.orange,
                        2: colors.blue
                    }[analysis_reliability[i]],
                    horizontally_center=True
                )
                
                added_lines_so_far_in_this_page += 1

    cover_pdf.save()

# 15. Join the PDF chunks and add the cover

In [ ]:
if params['generate_report']:

    aux_path_to_report_pdf = os.path.join(
        summary_dir,
        params['report_output_filename']
    )

    if params['verbose']:
        print(
            f"Saving the report PDF to "
            f"{os.path.abspath(aux_path_to_report_pdf)}"
        )

    PDFGenerator.concatenate_PDFs(
        pdf_reports_filepaths,
        aux_path_to_report_pdf
    )

    PDFGenerator.add_cover(
        path_to_cover_pdf,
        aux_path_to_report_pdf,
        # Overwritting the report PDF with the
        # one which already includes the cover
        aux_path_to_report_pdf,
    )

    os.remove(path_to_cover_pdf)    
    for filepath in pdf_reports_filepaths:
        os.remove(filepath)

# Debugging / Trash can

In [ ]:
# Oscillatority analysis
from scipy import interpolate as spinter

def oscillatority(x, y, period):

    """
    - x (unidimensional numpy array)
    - y (unidimensional numpy array): x.shape must match y.shape
    - period (scalar float)
    """

    # Implementing oscillatority as given by
    # (\int _{t_0}^{t_0+\Delta t} (f(t)-mean_f)*(f(t+period)-mean_f) dt)/\Delta t

    # Determine actual interval of integration,
    # which is [x[0], x[-1]-period]


    f = spinter.CubicSpline(x, y)
    average_f = np.mean(y)

    mask = (x <= (x[-1]-period))
    reduced_x = x[mask]
    reduced_y = y[mask]

    integration_width = (x[-1]-period)-x[0]

    integrand = np.vectorize(lambda input : ((f(input)-average_f)*(f(input+period)-average_f))/integration_width)
    integrand = integrand(reduced_x)

    # Normalized by the square amplitude of the signal
    return np.trapz(integrand, x=reduced_x)/((np.max(y)-np.min(x))*(np.max(y)-np.min(x)))

In [ ]:
# darknoisemeas_objects[i].Waveforms.plot(
#     wvfs_to_plot=18, 
#     plot_peaks=False,
#     #ylim=(0.00, 0.18),
#     #ylim=(0.03,0.09),
#     wvf_linewidth=0.2,
#     x0=[],
#     y0=[],
#     randomize=True,
#     fig_title=f"Iterator {i}, {darknoisemeas_objects[i].get_title()}",
#     mode='grid',
#     nrows=3, ncols=3
# )